In [ ]:
# Colab: Mount into drive
from google.colab import drive
drive.mount("/content/drive")
#place this tutorial.ipynb in your google drive under below directories (of course you need to create these folders first!):
#/SideProjects/LLM/SgLang/
%cd '/content/drive/MyDrive/SideProjects/LLM/SgLang/'

In [ ]:
#import all necassary packages:
! pip install --upgrade pip
! pip install "sglang[all]"
! pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/
! pip install triton

In [ ]:
#log into hugging face: this is used to connect with the selected language model
!huggingface-cli login

In [ ]:
#used to running the server (you can choose any, but I recommend start with the parameter with 8b!) in the background, so that we can run the next cell in the colab
#we need to wait one or two minutes before running the next cell!
import subprocess
subprocess.Popen(['python', '-m', 'sglang.launch_server', '--model-path', 'meta-llama/Meta-Llama-3-8B-Instruct', '--port', '30000'])


In [ ]:
#check if the server is running or not:
!ps -aux | grep sglang.launch_server

In [ ]:
#this is an example to show the main component of sglang.

#all neccasarry packages to be used in sglang, you can add some others as well!
from sglang import function, system, user, assistant, gen, set_default_backend, RuntimeEndpoint

#this is just the framwork!
#we have the system about the background set up
# we have user, and it is used to give some questions
#we have assistant, this is used to generate some answers, and we can assign any variable as answer here!
@function
def multi_turn_question(s, question_1, question_2):
    s += system("You are a helpful assistant.")
    s += user(question_1)
    s += assistant(gen("answer_1", max_tokens=256))
    s += user(question_2)
    s += assistant(gen("answer_2", max_tokens=256))

#this is used to call the server and run it locally!
set_default_backend(RuntimeEndpoint("http://localhost:30000"))

#this is just call the function we had!
q1 = "What is the capital of the United States?"
q2 = "List two local attractions."
state = multi_turn_question.run(
    question_1=q1,
    question_2=q2,
)

#this is some printed results, and we can figure out what's going on for this variable?
print("\n========== all in raw format ==========\n")
print(state)

print("\n========== useful content ==========\n")
for m in state.messages():
    print(m)

print("\n========== in certain structure ==========\n")
for m in state.messages():
    print(m["role"], ":", m["content"])

print("\n========== question 1 with answer ==========\n")
print(q1 + '\n')
print(state["answer_1"])

print("\n========== question 2 with answer ==========\n")
print(q2 + '\n')
print(state["answer_2"])
